In [1]:
# %matplotlib
# %matplotlib inline
# %matplotlib notebook

import pandas as pd
import numpy as np
import random
import os
import warnings
from datetime import datetime, timedelta, timezone
warnings.simplefilter("ignore")
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import FormatStrFormatter
import re
import math
from copy import deepcopy
from numba_stats import t
import scipy.stats as stats

# Data
from gridmeter import Data
from gridmeter import Data_Settings

# IMM
from gridmeter import IMM
from gridmeter import IMM_Settings

# Clustering
from gridmeter import Clustering
from gridmeter import Clustering_Settings

from IPython.display import Image, Markdown, display
plt.ion()
plt.rcParams['figure.figsize'] = [24, 16]
plt.rcParams['figure.dpi'] = 300

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

%load_ext autoreload
%autoreload 2

In [2]:
df_ls_t = pd.read_csv("/app/.recurve_cache/clustering/example_dfs/df_ls_t.csv")
df_ls_cp = pd.read_csv("/app/.recurve_cache/clustering/example_dfs/df_ls_cp.csv")

df_ls_t = df_ls_t.rename(columns={"hour": "time", "ls": "loadshape"})
df_ls_cp = df_ls_cp.rename(columns={"hour": "time", "ls": "loadshape"})

treatment_data = Data()
treatment_data.set_data(df_ls_t)

comparison_pool_data = Data()
comparison_pool_data.set_data(df_ls_cp)

In [3]:
# Test Data with ts input

# Create a testing dataframe having an id, datetime of 15 min intervals, observed and modeled values 
num_intervals = 4 * 24 * 365  # 4 intervals/hour * 24 hours/day * 365 days

# Create a DataFrame with 'id', 'datetime', 'observed', and 'modeled' columns
df = pd.DataFrame({
    'id': np.repeat(['id1', 'id2', 'id3'], num_intervals),  # only 3 ids for easier comparison
    'datetime': pd.date_range(start='2023-01-01', periods=num_intervals, freq='15T').tolist() * 3, 
    'observed': np.random.rand(num_intervals * 3),  # randomized
    'modeled': np.random.rand(num_intervals * 3)  # randomized
})

df_features = pd.DataFrame({
    'id': ['id1', 'id2', 'id3'],  # only 3 ids for easier comparison
    'feature1': np.random.rand(3),  # randomized
    'feature2': [None, *np.random.rand(2)]  # randomized
    # 'feature2': *np.random.rand(3)  # randomized
})

data = Data()
data.set_data(time_series_df=df, features_df=df_features)
data.loadshape
# data.features
data.excluded_ids

,id,reason
0,id1,null values in features_df


In [4]:
# Test IMM

imm_settings = IMM_Settings()
df_cg, df_t_coeffs = IMM(imm_settings).get_comparison_group(treatment_data, comparison_pool_data)

In [ ]:
from gridmeter._utils.calculate_distances import calculate_distances
from copy import deepcopy as copy

def TestDistanceMatching(
    df_ls_t,
    df_ls_c,
    n_matches_per_treatment=4,
    distance_metric="euclidean",
    allow_duplicate_match=True,
    replace_duplicate_method=None,  # currently unused [None, "closest_to_meter", "closest_global"]
    max_distance_threshold=None,
    n_match_multiplier=None,
    n_meters_per_chunk=10000,
):
    ls_t = df_ls_t.to_numpy()
    ls_cp = df_ls_c.to_numpy()

    n_matches_per_chunk = copy(n_matches_per_treatment)

    # Calculate closest distances
    if n_match_multiplier is None:
        n_matches_per_chunk = None

    if n_match_multiplier is not None:
        if (not allow_duplicate_match and replace_duplicate_method is not None) or max_distance_threshold is not None:
            n_matches_per_chunk *= n_match_multiplier

        if n_matches_per_chunk > ls_cp.shape[0]:
            n_matches_per_chunk = ls_cp.shape[0]

    cp_id_idx, dist = calculate_distances(
        ls_t, ls_cp, distance_metric, n_matches_per_chunk, n_meters_per_chunk
    )

    print(dist.shape)
    print(cp_id_idx.shape)

    # create dataframes
    id_t = df_ls_t.index.values
    id_c = df_ls_c.index.values

    print(np.repeat(id_t, dist.shape[1]))

    series_t = pd.Series(np.repeat(id_t, dist.shape[1]), name="treatment")
    series_cp = pd.Series(id_c[cp_id_idx.flatten()], name="id")
    clusters = pd.DataFrame(
        dist.flatten(), index=[series_t, series_cp], columns=["distance"]
    )
    clusters = clusters.reset_index()
    clusters["duplicated"] = clusters.duplicated(subset=["id"])
    clusters["cluster"] = 1
    clusters = clusters.set_index("id")
    
    if allow_duplicate_match:
        clusters = clusters.sort_values(by=["treatment", "distance"])

        # for each index, get 4 smallest distances
        clusters = clusters.groupby("treatment").head(n_matches_per_treatment)

    else:
        # get count of treatment and id pairs
        # t_id_counts = clusters.groupby(["treatment", "id"]).size()
        # print(t_id_counts)

        # drop duplicate index
        clusters = clusters[~clusters.index.duplicated(keep='first')]

        if replace_duplicate_method is not None:
            raise NotImplementedError(
                "'replace_duplicate_meters': True not implemented"
            )

    return clusters


def get_comparison_group(df_ls_t, df_ls_cp, weights=None, **kwargs):
    df_cg = TestDistanceMatching(df_ls_t, df_ls_cp, **kwargs)

    # Create df_t_coeffs
    t_ids = df_ls_t.index.unique()
    coeffs = np.ones(t_ids.values.size)

    df_t_coeffs = pd.DataFrame(coeffs, index=t_ids, columns=["pct_cluster_1"])
    df_t_coeffs.index.name = "id"

    return df_cg, df_t_coeffs

In [ ]:
df_cg, df_t_coeffs = get_comparison_group(df_ls_t_mod, df_ls_cp_mod, allow_duplicate_match=True, n_match_multiplier=2)
df_cg.reset_index().sort_values(by=["treatment", "id", "distance"])

In [ ]:
df_cg.sort_values(["treatment", "distance"])

In [ ]:
df_ls_t

In [ ]:
df_ls_t_mod.stack().reset_index().rename(columns={0: "ls"})

In [13]:
# Test Clustering

clustering_settings = Clustering_Settings()
df_cg, df_t_coeffs = Clustering(clustering_settings).get_comparison_group(treatment_data, comparison_pool_data)
df_cg

,cluster
id,
None-6500609087-6500609087,0
None-4394605605-4394605605,0
None-1572272305-1572272305,0
None-2615141910-2615141910,0
None-2615294205-2615294205,0
...,...
None-3590533705-3590533705,8
None-1721108410-1721108410,8
None-1709262405-1709262405,8


In [14]:
df_t_coeffs

,pct_cluster_0,pct_cluster_1,pct_cluster_2,pct_cluster_3,pct_cluster_4,pct_cluster_5,pct_cluster_6,pct_cluster_7
id,,,,,,,,
None-1094275585-1094275585,2.163893e-12,5.029705e-01,4.288347e-01,0.000000e+00,4.847305e-13,0.000000e+00,4.097588e-02,2.721895e-02
None-1397301805-1397301805,1.574652e-11,1.887717e-01,6.672347e-01,1.439936e-01,7.034535e-12,0.000000e+00,1.078794e-11,0.000000e+00
None-1432022910-1432022910,1.296811e-12,6.902494e-04,4.342816e-03,7.915817e-05,1.381838e-12,3.685489e-06,3.904629e-01,6.044212e-01
None-1469355610-1469355610,4.351317e-12,3.474865e-01,5.179979e-01,1.345155e-01,3.327867e-12,0.000000e+00,1.160953e-14,1.171110e-12
None-1504812305-1504812305,7.987435e-12,4.239205e-01,4.190504e-01,1.570292e-01,1.727044e-12,0.000000e+00,1.716741e-11,0.000000e+00
...,...,...,...,...,...,...,...,...
None-5519977972-5519977972,3.001672e-05,1.697287e-07,5.576015e-02,1.722308e-03,3.799684e-02,2.551940e-01,3.890770e-01,2.602195e-01
None-8098217928-8098217928,7.050122e-08,3.575080e-01,4.697889e-01,9.410056e-03,1.201133e-01,6.750067e-03,3.872386e-03,3.255729e-02
None-8313277985-8313277985,2.632192e-12,1.684967e-02,1.464463e-11,8.571292e-11,1.276320e-11,2.251083e-01,3.401503e-11,7.580420e-01
